In [1]:
import os
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from NN_config import Config
from HD_DataLoader import *
from NN_model import *


Get the database:

In [2]:
folder_path = os.path.join(os.getcwd(), 'HD_data') 
HD_class = HD(data_path = folder_path)
database = HD_class.generate_database()

---------------------------------------------------------
Generating database for HD's


Get the testing, training and validation set:

In [14]:
# Assuming you have a database variable containing your raw data
dataset = VairableSensorsDataset(database, mic_bool='True', vibr_bool='False', cur_bool='False', start_percentage=0.1, stop_percentage = 0.9, window_sec = 2, stride_sec=0.5)

# Splitting the data into train, test, and validation sets
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.1, random_state=42)

# Creating data loaders
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


Number of inputs is 1170
Number of labels is 1170


Start the training of the model

In [15]:
train_features, train_labels = next(iter(train_dataloader))
input_dim = train_features.shape[0]

cfg = Config()

# Initialize network
model = Classifier(input_dim)
model.train()
if cfg.enable_cuda:
    model = model.cuda()

# Initialize optimizer
# optimizer = optim.SGD(model.parameters(), lr=cfg.lr, momentum=cfg.lr_momentum, weight_decay=cfg.weight_decay)
optimizer = optim.Adam(model.parameters(), lr=cfg.lr)

criterion = nn.CrossEntropyLoss()

# Loop over inputs
running_loss = 0.0
i = 0

for epoch in range(cfg.num_epochs):
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        print(outputs.shape, labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print("Finished training.")
save_path = 'model.pth'
torch.save(model.state_dict(), save_path)
print("Saved trained model as {}.".format(save_path))

RuntimeError: stack expects each tensor to be equal size, but got [1] at entry 0 and [0] at entry 1

Hugo his code using different classes in the dataloader

In [ ]:
# test = SplitNormalFaulty(database)
# working, faulty = test.split_faulty_working_vib('X_vibr')
# Data_prep = DataPreprocessing(working, faulty)

# inputs, labels = Data_prep.create_data_batches()

# print("Number of inputs is {}".format(len(inputs)))


In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.4, random_state=42)
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

# training_data = CustomDataset(x_train, y_train)
# test_data = CustomDataset(x_test, y_test)
# val_data = CustomDataset(x_val, y_val)

# # training_data = CustomImageDataset(x_train, y_train)
# # test_data = CustomImageDataset(x_test, y_test)

# train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
# test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
# test_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)

In [ ]:
# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")